In [1]:
import math
import numpy as np

# Elliptical, parabolic, and hyperbolic orbits using the Euler method

In the simple_two_body.ipynb notebook I implemented circular orbits with arbitary starting positions and confirmed that the truncation error converged linearly with timestep for the Euler method.  

In this notebook I plan to implement hyperbolic, elliptical, and parabolic orbits using the Euler method. Parabolic orbits are marginally bound, hyperbolic orbits are unbound, and elliptical orbits are bound but oblong. All orbits should have the center of mass at one focus for both stars orbit and be symmetric between the two stars. 

Elliptical orbits should be elongated away from the other star. 

Hyperbolic and parabolic orbits should open away from the other star. 

All orbits should conserve energy and angular momentum. Later this could be enforced. I will probably not do that in this notebook. 

It would be meaningless to evaluate the L0 error for a hyperbolic or parabolic orbit since there is no cycle and the L0 error is found after one or more cycles when the system returns to its starting point. However, it would be possible to evaluate it for the elliptical orbit system, and I intend to check that the convergence of the L0 error with dt is linear as it should be for the Euler method. 

Because the convergence can be tested for ellipses only and because ellipses are closed so make it clear if they are not working correctly graphically, I will begin with ellipses then proceed to parabolas and hyperbolas. 

# Conic sections

Circles, ellipses, parabolas, and hyperbolas are called conic sections because they can all be obtained by taking slices of an infinite cone. 

Conic sections are related to each other through a parameter called eccentricity. Circles have an eccentricity of 0. Ellipses have an eccentricity of 0<e<1. Parabolas have an eccentricity of e=1. Hyperbolas have an eccentricity of e>1. 

Circles are described by the equation x^2+y^2=r^2 where r is the radius. They can also be described parametrically using sine and cosine. That is done below. 

In [2]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
radius=10
t=2*math.pi*0.01*np.arange(103)
centerx=3
centery=5
p = figure(title="Circle", plot_width=400, plot_height=400)
p.line(centerx+np.cos(t),centery+np.sin(t),line_width=3,line_color="blue",legend="Circle")
p.circle(centerx,centery,line_width=15,line_color="green",legend="Center")
p.legend.location="bottom_right"
show(p)

Loading BokehJS ...

Ellipses are shaped like ovals. They have two locations called focuses that determine their shapes. If a string were connected to both focuses, which are inside the ellipse, you could slide a pencil along the string and draw the ellipse keeping the string taut. The two focuses and the ellipse form a triangle with hypoteneuses of fixed length. 

Half the shorter of the two widths of the ellipse is called the semimajor axis, a. It's like a radius for ellipses. The lenght of the longer width is (1+e)*a. The focus is offset from the center by f=e*sqrt(1-e^2)*a. 

Ellipses can also be drawn at angles to the axis. An ellipse aligned with the x and y axis is shown below. 

The nonparametric equation for an ellipse is x^2/a^2+y^2/b^2=1
Below the ellipse is described parametrically. 

In [3]:
def getfocus(a,ecc):
    b=a*(1+ecc)
    return np.sqrt(b**2-a**2)

In [4]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
radius=10
t=2*math.pi*0.01*np.arange(103)
a=10
ecc=.5
b=a*(1+ecc)
centerx=0
centery=0
focusx=centerx+getfocus(a,ecc)
focusy=centery+0.0
p = figure(title="Ellipse", plot_width=400, plot_height=400, x_range=[-15,15], y_range=[-15,15])
p.line(centerx+b*np.cos(t),centery+a*np.sin(t),line_width=3,line_color="blue",legend="Circle")
p.circle(focusx,focusy,line_width=15,line_color="green",legend="Focus 1")
p.circle(centerx-getfocus(a,ecc),focusy, line_width=15, line_color="red", legend="Focus 2")
p.legend.location="bottom_right"
show(p)

Loading BokehJS ...

In [5]:
import numpy as np
import matplotlib.pyplot as plt

def NewtonianForce(mass1,mass2, r):
    return (6.67408*10**-11)*mass1*mass2/r/r;

In [6]:
def euler(h,t, x,y,z,f,debugprint):
    xstep = (f(t, x, y,z)+f(t+h,x+h,y,z))/2*h
    xnew=x+xstep
    if(debugprint):
        print(h,t,x,xstep,xnew)
    return t+h,xnew

In [21]:
import math
def InitialDataEqualMassConic(radius,ecc,angle,initmass):
    orbitalangle=angle
    print("angle",angle)
    phi=np.array([math.pi+orbitalangle,orbitalangle])
    orbitalradius=radius*np.ones(2) #semimajor axis
    eccentricity=ecc
    print(eccentricity)
    mass=np.ones(2)
    masses=initmass*mass #*masssun (natural units)
    return phi,orbitalangle,orbitalradius,eccentricity, masses
    

In [66]:
import random,numpy as np
rad0=100
ecc0=0.3
angle0=0.0
mass0=1.0
initdateqellipse=InitialDataEqualMassConic(rad0,ecc0,angle0,mass0)
print(initdateqellipse)

TypeError: InitialDataEqualMassConic() takes 3 positional arguments but 4 were given

In [67]:
def getxyuveqellipseredmass(initdat):
    phi,orbitangle,orbitalradius,eccentricity, masses=initdat
    print("initdat", orbitalradius, phi, np.cos(phi), np.sin(phi))
    metersperAU=1
    Gconstant=1
    #fix x0 y0 at one star, disregard initial data, use orbital radius as separation between stars
    #this is consistant with choice in previous part
    cosphi=np.cos(phi)
    sinphi=np.sin(phi)
    coordsep=orbitalradius #/2.
    print("coordsep",coordsep)
    #x0=orbitalradius/2.*np.cos(phi)*metersperAU
    #y0=orbitalradius/2.*np.sin(phi)*metersperAU
    count=0
    for phi0 in phi:
        if phi0==0:
            print("zero")
            #x0[count]=orbitalradius[count]/2.
            #y0[count]=0
            cosphi[count]=1.0
            sinphi[count]=0.0
        if phi0==math.pi:
            cosphi[count]=-1.0
            sinphi[count]=0.0
            print("pi")
            #x0[count]=-orbitalradius[count]/2.
            #y0[count]=0
        if phi0==math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=1.0
            #x0[count]=0
            #y0[count]=orbitalradius[count]/2.
        if phi0==3*math.pi/2.:
            cosphi[count]=0.0
            sinphi[count]=-1.0
            #x0[count]=0
            #y0[count]=-orbitalradius[count]/2.
        count+=1
    x0=coordsep*cosphi
    y0=coordsep*sinphi
    #x0[1]=0.0
    #y0[1]=0.0
    z0=np.zeros(2)
    print(x0)
    print(y0)
    v=np.zeros(2)
    a=np.zeros(2)
    ax0=0.
    ay0=0.
    az0=0.
    vx0=0.
    vy0=0.
    vz0=0.
    print(eccentricity)
    if eccentricity<1 and eccentricity>0:
        #elliptical
        orbitalr=orbitalradius[0]/2. #reduced mass
        focusdisp=getfocus(orbitalr,eccentricity) #displacement of the center due to the focus being at the center
        E=-Gconstant*masses[0]*masses[1]/(2*orbitalr) #orbitalradius is correct here r1+r2.
        #This factor of half is due to the virial (VEE-REE-AHL) theorem, a stellar dynamics theorem governing the 
        #equipartion of degrees of freedom of energy in a system. Another way of writing it is T=-1/2*V. 2T-V=0
        coordsep=orbitalr*(1.+eccentricity) #starts at apastron
        starsep=coordsep #reduced mass, M is at center
        Eapastron=-Gconstant*masses[0]*masses[1]/(2*starsep) #virial theorem again 
        Fapastron=Eapastron/(starsep)
        print("E,F",E,Eapastron,2*Eapastron, 2*orbitalr,starsep)
        x0=(coordsep+focusdisp)*cosphi
        y0=(coordsep+focusdisp)*sinphi
        x0[0]=0.0
        y0[0]=0.0
        v=np.zeros(2)
        v[0]=0.0
        v[1]=np.sqrt(2.*(E-2.*Eapastron)/masses[1]) #1/2*mv^2-GMm/starsep=E. Solve for v. Eapastron=-GMm/2starsep
        ux0=-v*sinphi
        uy0=v*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
        a=np.zeros(2)
        a[0]=0.0
        a[1]=Fapastron/masses[1]
    
        ax0=-a*cosphi
        ay0=-a*sinphi
        az0=np.zeros(2)
    elif eccentricity[0]==0.0: #circular
        #start at perihelion for both (eliptical, doesn't generalize to three body)
        #actually start with circular orbit
        ux0=np.zeros(2) #*149597870700
        #centrepital force balances gravitational force
        metersperAU=1 #natural units
        #G=1
        Gconstant=1
        #Fcentripital=mass1*v**2/rphys
        #centF=accel
        #Faccel=G*m1*m2/r^2
        r0=2.*orbitalradius #Mystery factor of 2
        print("r0", r0)
        v=np.zeros(2)
        for i in np.arange(2):
            v[i]=np.sqrt(Gconstant*masses[(i+1)%2]/np.abs(r0[i]))
        print(v)
        #r0=orbitalradius #np.sqrt(x0**2+y0**2)
        ux0=-v*sinphi
        uy0=v*cosphi #initial data in y only 
        uz0=np.zeros(2)
    
    a=np.zeros(2)
    a[0]=0.0
    a[1]=Fapastron/reducedmass[1]
    
    ax0=-a*cosphi
    ay0=-a*sinphi
    az0=np.zeros(2)

    
    
    return masses,x0,y0,z0, ux0, uy0,uz0, ax0, ay0,az0

In [68]:
xyuvaeqellipseredmass=getxyuveqellipseredmass(initdateqellipse)
print(xyuvaeqellipseredmass)

initdat [100. 100.] [3.14159265 0.        ] [-1.  1.] [1.2246468e-16 0.0000000e+00]
coordsep [100. 100.]
pi
zero
[-100.  100.]
[0. 0.]
0.3
E,F -0.01 -0.007692307692307693 -0.015384615384615385 100.0 65.0
(array([1., 1.]), array([  0.        , 106.53311931]), array([0., 0.]), array([0., 0.]), array([-0., -0.]), array([-0.       ,  0.1037749]), array([0., 0.]), array([0.        , 0.00011834]), array([-0.,  0.]), array([0., 0.]))


In [69]:
def timestep(step,t,dt,mtotal, mass,xi,yi,zi, vxi, vyi, vzi, axi, ayi, azi):
    xii=np.zeros(np.size(xi))
    vxii=np.zeros(np.size(vxi))
    yii=np.zeros(np.size(yi))
    vyii=np.zeros(np.size(vyi))
    zii=np.zeros(np.size(vzi))
    vzii=np.zeros(np.size(vzi))
    rii=np.zeros(np.size(xi))
    axii=np.zeros(np.size(axi))
    ayii=np.zeros(np.size(ayi))
    azii=np.zeros(np.size(azi))
    
    #This is redundant in the center of mass system
    Gconstant=6.408*10**-11
    Gconstant=1
    for k in np.arange(len(rii)):
        for j in np.arange(len(rii)):
            if j!=k:
                if j==0: #reduced mass
                    axii[0]=0.0
                    ayii[0]=0.0
                    azii[0]=0.0
                else:
                    rreljk=np.abs((xi[j] - xi[k])**2+(yi[j]-yi[k])**2+(zi[j]-zi[k])**2)**(1./2.)
                    #print(rreljk)
                    axii[j]-=Gconstant*mass[k]*(xi[j]  - xi[k])/rreljk**3
                    ayii[j]-=Gconstant*mass[k]*(yi[j]  - yi[k])/rreljk**3
                    azii[j]-=Gconstant*mass[k]*(zi[j]  - zi[k])/rreljk**3
    #print(xii)
    
    for m in np.arange(len(x)):
        #m represents choices of mass
        i=step
        
        xii[m] = xi[m] + dt*vxi[m]
        vxii[m] = vxi[m] + dt*axi[m]
        yii[m]= yi[m] + dt*vyi[m]
        vyii[m] = vyi[m] + dt*ayi[m]
        zii[m]= zi[m] + dt*vzi[m]
        vzii[m] = vzi[m] + dt*azi[m]
        rii[m]=np.sqrt(xi[m]**2+yi[m]**2+zi[m]**2)
        
        #print(vyii[m],vyi[m],ayi[m],dt)
        #print(yii[m],yi[m],vyi[m],dt)
    
    
    return mass, xii,yii,zii,vxii,vyii,vzii,axii,ayii,azii
                    

In [70]:
dt=1 #*31556926 #seconds per year
numsteps=10000
mass0,x,y,z0,vx,vy,vz0,ax,ay,az0=xyuvaeqellipseredmass
print(x)
xcoord1=[]
xcoord2=[]
ycoord1=[]
ycoord2=[]
t=0.0
masstotal=1.
for i in np.arange(1,numsteps):
    mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(i,t,dt,masstotal,mass0,x,y,z0,vx,vy,vz0,ax,ay,az0)
    print(x,y,vx,vy,ax,ay)
    #print(ay) #forces should be equal and opposite, but in reduced mass framework accelerations are also equal and opposite
    #accelerations should evolve from y to x with time in a sinusoidal manner even in reduced mass framework
    #print(ax)
    xcoord1.append(x[0])
    xcoord2.append(x[1])
    ycoord1.append(y[0])
    ycoord2.append(y[1])
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(2,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(3,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 
#mass, x,y,z,vx,vy,vz,ax,ay,az=timestep(4,0,dt,mass,x,y,z,vx,vy,vz,ax,ay,az)
#print(x,y,vx,vy,ax,ay) 

[  0.         106.53311931]
[  0.         106.53311931] [0.        0.1037749] [0.         0.00011834] [-0.         0.1037749] [ 0.00000000e+00 -8.81111183e-05] [0. 0.]
[  0.         106.53323766] [0.         0.20754981] [0.0000000e+00 3.0232077e-05] [0.        0.1037749] [ 0.00000000e+00 -8.81109928e-05] [ 0.00000000e+00 -8.58297393e-08]
[  0.         106.53326789] [0.         0.31132471] [ 0.00000000e+00 -5.78789158e-05] [0.         0.10377482] [ 0.00000000e+00 -8.81104209e-05] [ 0.00000000e+00 -1.71658173e-07]
[  0.         106.53321001] [0.         0.41509953] [ 0.         -0.00014599] [0.         0.10377465] [ 0.00000000e+00 -8.81097438e-05] [ 0.00000000e+00 -2.57485209e-07]
[  0.         106.53306402] [0.         0.51887418] [ 0.        -0.0002341] [0.         0.10377439] [ 0.00000000e+00 -8.81089617e-05] [ 0.0000000e+00 -3.4331068e-07]
[  0.         106.53282992] [0.         0.62264857] [ 0.         -0.00032221] [0.         0.10377405] [ 0.00000000e+00 -8.81080745e-05] [ 0.000000

[  0.         105.13981717] [ 0.         18.70151328] [ 0.         -0.01563438] [0.        0.1024003] [ 0.00000000e+00 -8.63514773e-05] [ 0.00000000e+00 -1.52732061e-05]
[  0.         105.12418278] [ 0.         18.80391358] [ 0.         -0.01572073] [0.         0.10238503] [ 0.00000000e+00 -8.63324582e-05] [ 0.00000000e+00 -1.53561957e-05]
[  0.         105.10846205] [ 0.         18.90629861] [ 0.         -0.01580707] [0.         0.10236967] [ 0.00000000e+00 -8.63133397e-05] [ 0.00000000e+00 -1.54391553e-05]
[  0.         105.09265498] [ 0.         19.00866828] [ 0.         -0.01589338] [0.         0.10235423] [ 0.0000000e+00 -8.6294122e-05] [ 0.00000000e+00 -1.55220846e-05]
[  0.         105.07676161] [ 0.         19.11102252] [ 0.         -0.01597967] [0.         0.10233871] [ 0.00000000e+00 -8.62748051e-05] [ 0.00000000e+00 -1.56049835e-05]
[  0.         105.06078193] [ 0.         19.21336123] [ 0.         -0.01606595] [0.         0.10232311] [ 0.00000000e+00 -8.62553891e-05] [ 0.00

[  0.         102.17555298] [ 0.         32.65330541] [ 0.         -0.02732952] [0.        0.0995231] [ 0.00000000e+00 -8.28178285e-05] [ 0.00000000e+00 -2.63792322e-05]
[  0.         102.14822347] [ 0.         32.75282852] [ 0.         -0.02741233] [0.         0.09949673] [ 0.00000000e+00 -8.27857906e-05] [ 0.00000000e+00 -2.64567171e-05]
[  0.         102.12081113] [ 0.         32.85232524] [ 0.         -0.02749512] [0.         0.09947027] [ 0.00000000e+00 -8.27536643e-05] [ 0.00000000e+00 -2.65341529e-05]
[  0.         102.09331601] [ 0.         32.95179551] [ 0.         -0.02757787] [0.         0.09944373] [ 0.00000000e+00 -8.27214496e-05] [ 0.00000000e+00 -2.66115392e-05]
[  0.         102.06573814] [ 0.         33.05123924] [ 0.        -0.0276606] [0.         0.09941712] [ 0.00000000e+00 -8.26891466e-05] [ 0.0000000e+00 -2.6688876e-05]
[  0.         102.03807754] [ 0.         33.15065637] [ 0.         -0.02774328] [0.         0.09939043] [ 0.00000000e+00 -8.26567554e-05] [ 0.0000

[ 0.         96.49067188] [ 0.         48.85061125] [ 0.         -0.04065177] [0.         0.09408498] [ 0.00000000e+00 -7.63204671e-05] [ 0.00000000e+00 -3.85483253e-05]
[ 0.         96.45002011] [ 0.         48.94469623] [ 0.         -0.04072809] [0.         0.09404644] [ 0.00000000e+00 -7.62751294e-05] [ 0.00000000e+00 -3.86160302e-05]
[ 0.         96.40929201] [ 0.         49.03874267] [ 0.         -0.04080437] [0.         0.09400782] [ 0.00000000e+00 -7.62297218e-05] [ 0.00000000e+00 -3.86836682e-05]
[ 0.         96.36848764] [ 0.         49.13275049] [ 0.        -0.0408806] [0.         0.09396914] [ 0.00000000e+00 -7.61842443e-05] [ 0.00000000e+00 -3.87512394e-05]
[ 0.         96.32760705] [ 0.         49.22671962] [ 0.         -0.04095678] [0.         0.09393038] [ 0.00000000e+00 -7.61386972e-05] [ 0.00000000e+00 -3.88187436e-05]
[ 0.         96.28665026] [ 0.         49.32065001] [ 0.         -0.04103292] [0.         0.09389157] [ 0.00000000e+00 -7.60930805e-05] [ 0.00000000e+00

[ 0.         89.30788846] [ 0.         62.93409698] [ 0.         -0.05190069] [0.         0.08734433] [ 0.00000000e+00 -6.85354669e-05] [ 0.00000000e+00 -4.82010039e-05]
[ 0.         89.25598778] [ 0.         63.02144131] [ 0.         -0.05196922] [0.         0.08729613] [ 0.00000000e+00 -6.84808098e-05] [ 0.0000000e+00 -4.8257528e-05]
[ 0.         89.20401856] [ 0.         63.10873744] [ 0.        -0.0520377] [0.         0.08724787] [ 0.00000000e+00 -6.84261027e-05] [ 0.00000000e+00 -4.83139756e-05]
[ 0.         89.15198085] [ 0.         63.19598531] [ 0.         -0.05210613] [0.         0.08719956] [ 0.00000000e+00 -6.83713457e-05] [ 0.00000000e+00 -4.83703467e-05]
[ 0.         89.09987473] [ 0.         63.28318486] [ 0.        -0.0521745] [0.         0.08715119] [ 0.00000000e+00 -6.83165391e-05] [ 0.00000000e+00 -4.84266413e-05]
[ 0.         89.04770023] [ 0.         63.37033605] [ 0.         -0.05224282] [0.         0.08710276] [ 0.00000000e+00 -6.82616829e-05] [ 0.00000000e+00 -4.

...

[ 0.         98.41661107] [ 0.         48.96749041] [ 0.        -0.0397302] [0.         0.09368644] [ 0.00000000e+00 -7.41344266e-05] [ 0.00000000e+00 -3.68003868e-05]
[ 0.         98.37688087] [ 0.         49.06117686] [ 0.         -0.03980433] [0.         0.09364964] [ 0.00000000e+00 -7.40920309e-05] [ 0.000000e+00 -3.686472e-05]
[ 0.         98.33707654] [ 0.        49.1548265] [ 0.         -0.03987842] [0.         0.09361278] [ 0.00000000e+00 -7.40495699e-05] [ 0.0000000e+00 -3.6928992e-05]
[ 0.         98.29719812] [ 0.         49.24843928] [ 0.         -0.03995247] [0.         0.09357585] [ 0.00000000e+00 -7.40070437e-05] [ 0.00000000e+00 -3.69932026e-05]
[ 0.         98.25724565] [ 0.         49.34201513] [ 0.         -0.04002648] [0.         0.09353886] [ 0.00000000e+00 -7.39644524e-05] [ 0.00000000e+00 -3.70573517e-05]
[ 0.         98.21721917] [ 0.         49.43555398] [ 0.         -0.04010044] [0.        0.0935018] [ 0.00000000e+00 -7.39217962e-05] [ 0.00000000e+00 -3.712143

[ 0.         90.53029177] [ 0.         64.48900142] [ 0.         -0.05180743] [0.         0.08646418] [ 0.00000000e+00 -6.59773266e-05] [ 0.00000000e+00 -4.69089067e-05]
[ 0.         90.47848433] [ 0.        64.5754656] [ 0.         -0.05187341] [0.         0.08641728] [ 0.00000000e+00 -6.59253935e-05] [ 0.00000000e+00 -4.69617706e-05]
[ 0.         90.42661093] [ 0.         64.66188288] [ 0.         -0.05193933] [0.         0.08637031] [ 0.0000000e+00 -6.5873415e-05] [ 0.00000000e+00 -4.70145635e-05]
[ 0.         90.37467159] [ 0.         64.74825319] [ 0.         -0.05200521] [0.        0.0863233] [ 0.00000000e+00 -6.58213915e-05] [ 0.00000000e+00 -4.70672854e-05]
[ 0.         90.32266638] [ 0.         64.83457649] [ 0.         -0.05207103] [0.         0.08627623] [ 0.0000000e+00 -6.5769323e-05] [ 0.00000000e+00 -4.71199364e-05]
[ 0.         90.27059535] [ 0.         64.92085272] [ 0.        -0.0521368] [0.         0.08622911] [ 0.00000000e+00 -6.57172096e-05] [ 0.00000000e+00 -4.7172

[ 0.       81.929097] [ 0.         77.06443254] [ 0.         -0.06121659] [0.         0.07876868] [ 0.00000000e+00 -5.76326202e-05] [ 0.00000000e+00 -5.41147526e-05]
[ 0.         81.86788041] [ 0.         77.14320122] [ 0.         -0.06127422] [0.         0.07871456] [ 0.00000000e+00 -5.75751723e-05] [ 0.00000000e+00 -5.41565591e-05]
[ 0.         81.80660618] [ 0.         77.22191578] [ 0.        -0.0613318] [0.         0.07866041] [ 0.00000000e+00 -5.75176971e-05] [ 0.00000000e+00 -5.41982919e-05]
[ 0.         81.74527438] [ 0.         77.30057619] [ 0.         -0.06138932] [0.         0.07860621] [ 0.00000000e+00 -5.74601946e-05] [ 0.0000000e+00 -5.4239951e-05]
[ 0.         81.68388507] [ 0.        77.3791824] [ 0.         -0.06144678] [0.         0.07855197] [ 0.0000000e+00 -5.7402665e-05] [ 0.00000000e+00 -5.42815363e-05]
[ 0.         81.62243829] [ 0.         77.45773437] [ 0.         -0.06150418] [0.         0.07849769] [ 0.00000000e+00 -5.73451084e-05] [ 0.0000000e+00 -5.4323048

[ 0.         74.72784269] [ 0.        85.5410764] [ 0.         -0.06730881] [0.         0.07246475] [ 0.00000000e+00 -5.10540221e-05] [ 0.00000000e+00 -5.83395622e-05]
[ 0.         74.66053388] [ 0.         85.61354115] [ 0.         -0.06735987] [0.         0.07240641] [ 0.00000000e+00 -5.09942032e-05] [ 0.00000000e+00 -5.83731428e-05]
[ 0.         74.59317401] [ 0.         85.68594756] [ 0.         -0.06741086] [0.         0.07234803] [ 0.00000000e+00 -5.09343694e-05] [ 0.00000000e+00 -5.84066507e-05]
[ 0.         74.52576315] [ 0.         85.75829559] [ 0.        -0.0674618] [0.         0.07228963] [ 0.00000000e+00 -5.08745205e-05] [ 0.00000000e+00 -5.84400859e-05]
[ 0.         74.45830135] [ 0.         85.83058522] [ 0.         -0.06751267] [0.         0.07223119] [ 0.00000000e+00 -5.08146568e-05] [ 0.00000000e+00 -5.84734483e-05]
[ 0.         74.39078868] [ 0.        85.9028164] [ 0.         -0.06756349] [0.         0.07217271] [ 0.00000000e+00 -5.07547783e-05] [ 0.0000000e+00 -5.8

[ 0.         63.87121254] [ 0.         95.93173897] [ 0.         -0.07441791] [0.         0.06319041] [ 0.00000000e+00 -4.17852541e-05] [ 0.0000000e+00 -6.2645276e-05]
[ 0.         63.79679463] [ 0.         95.99492938] [ 0.         -0.07445969] [0.         0.06312776] [ 0.0000000e+00 -4.1724279e-05] [ 0.00000000e+00 -6.26680234e-05]
[ 0.         63.72233493] [ 0.         96.05805714] [ 0.         -0.07450142] [0.         0.06306509] [ 0.00000000e+00 -4.16633035e-05] [ 0.00000000e+00 -6.26907026e-05]
[ 0.         63.64783351] [ 0.         96.12112223] [ 0.         -0.07454308] [0.        0.0630024] [ 0.00000000e+00 -4.16023277e-05] [ 0.00000000e+00 -6.27133136e-05]
[ 0.         63.57329043] [ 0.         96.18412464] [ 0.         -0.07458468] [0.         0.06293969] [ 0.00000000e+00 -4.15413516e-05] [ 0.00000000e+00 -6.27358563e-05]
[ 0.         63.49870575] [ 0.         96.24706432] [ 0.         -0.07462623] [0.         0.06287695] [ 0.00000000e+00 -4.14803754e-05] [ 0.00000000e+00 -6.

In [72]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()
# create a new plot with default tools, using figure
p = figure(title="dt=1, Euler Method, Reduced Mass, e=0.3", plot_width=400, plot_height=400, y_range=[-150,150],x_range=[-150,150])
t = np.linspace(0, 2*math.pi, 100)
focusx=getfocus(0.5*rad0,ecc0)
b=0.5*rad0*(1+ecc)
print(b)
print(focusx)
# add a circle renderer with x and y coordinates, size, color, and alpha
p.circle(xcoord1,ycoord1, size=15, line_color="purple",fill_color="purple", fill_alpha=0.5, legend="M")
p.circle(xcoord2,ycoord2, size=1, line_color="blue", fill_color="blue", fill_alpha=0.5,legend="m")
p.line(-focusx+rad0*(1+ecc0)*np.cos(t),rad0*np.sin(t), line_color="green", legend="Ideal ellipse")
p.legend.location = "bottom_left"
p.legend.click_policy="hide"

show(p) # show the results

Loading BokehJS ...

83.06623862918075


This doesn't handle the mass conversion to reduced mass system correctly yet but I want to check it in while I have an orbit. 

Well the scale and eccentricity is wrong. That's literally what I just said. 

In [ ]:
What about a circle makes it evolve to an ellipse? Is the acceleration higher relative to the velocity or is the velocity higher relative to the acceleration? I am gong 

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from math import pi

u=1.     #x-position of the center
v=0.5    #y-position of the center
a=2.     #radius on the x-axis
b=1.5    #radius on the y-axis

t = np.linspace(0, 2*pi, 100)
plt.plot(focusx+65*np.cos(t) , 50*np.sin(t) )
plt.grid(color='lightgray',linestyle='--')
plt.show()